In [28]:
# pip install selenium
# pip install easyocr

In [29]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.options import Options
import time

In [30]:
# 브라우저 꺼짐 방지 옵션
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)

# 드라이버 생성
driver = webdriver.Chrome(options=chrome_options)

# 브라우저 사이즈
driver.set_window_size(1900,1000)

# 웹페이지가 로드될때까지 2초를 대기
driver.implicitly_wait(time_to_wait=2)

driver.get(url='https://tickets.interpark.com/')

In [31]:
import os

# personal.txt 파일에서 아이디와 비밀번호 읽어오기
def get_credentials(file_path):
    personal = {}
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=')
            personal[key] = value
    return personal

# personal.txt 파일 경로
file_path = 'personal.txt'

# 아이디와 비밀번호 가져오기
personal = get_credentials(file_path)
username = personal.get('username')
password = personal.get('password')

# 가져온 아이디와 비밀번호를 사용하여 작업 수행
# print(f"Username: {username}")
# print(f"Password: {password}")

# 여기서 WebDriver 실행 등 필요한 작업을 수행합니다.


In [32]:
# 로그인
driver.find_element(By.LINK_TEXT,'로그인').click()
userId = driver.find_element(By.ID, 'userId')
userId.send_keys(username)
userPwd = driver.find_element(By.ID, "userPwd")
userPwd.send_keys(password)
userPwd.send_keys(Keys.ENTER)
# 로그인버튼 클릭
# driver.find_element(By.ID, "btn_login").click()

In [33]:
# 사이트 이동 - 현재 서울이랜드
driver.get('https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07002&TeamCode=PS006')
# 키움 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07001&TeamCode=PB003
# 두산 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07001&TeamCode=PB004
# LCK = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07032&TeamCode=PE015
# 서울이랜드 = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07002&TeamCode=PS098
# 천안시티FC = https://ticket.interpark.com/Contents/Sports/GoodsInfo?SportsCode=07002&TeamCode=PS006

# 첫번째 경기 예매하기 버튼 클릭 CSS_SELECTOR
driver.find_element(By.CSS_SELECTOR, f".timeScheduleList .timeSchedule:nth-of-type({1}) .BtnColor_Y.btn1").click()

In [34]:
import easyocr

In [35]:
# 팝업 창으로 포커싱 이동
driver.switch_to.window(driver.window_handles[1]) # 기존에 열려있던 창이 0, 팝업창이 1
time.sleep(0.5)
# 대기순서 창이 나올 수 있음

In [36]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [37]:
# 보안문자 iframe 창이 나오는 대기순서 class="wrap" 기다리기
wait = WebDriverWait(driver, 100)
wait.until(EC.presence_of_element_located((By.ID, "ifrmSeat")))

# ifrmSeat iframe으로 전환
ifrmSeat = driver.find_element(By.XPATH, '//iframe[@id="ifrmSeat"]')
driver.switch_to.frame(ifrmSeat)
time.sleep(0.5)

In [38]:
# easyocr 이미지 내에 인식할 언어 지정
reader = easyocr.Reader(['en'])

# 보안문자 입력 함수
def process_captcha():
    # 보안 문자 이미지 캡쳐
    captchaPng = driver.find_element(By.XPATH, "//*[@id='imgCaptcha']")

    # 캡쳐한 이미지에서 문자열 인식하기
    result = reader.readtext(captchaPng.screenshot_as_png, detail=0)

    # 이미지에 점과 직선이 포함되어 있어서 문자 인식이 완벽하지 않아 데이터를 수동으로 보정해줌
    captchaValue = result[0].replace('o', '0').replace('s', '5').replace('O', '0').replace('S', '5').replace(' ', '') \
        .replace('7.', '7').replace('4', '4').replace('1', '1').replace('(', '').replace(')', '') \
        .replace('l', 'L').replace('e', '9').replace('S', '5').replace('C', 'C').replace('L', 'L').replace('.', '')

    print(captchaValue)

    # 유효성 텍스트 박스 찾기
    validation_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'validationTxt')]")
    for element in validation_elements:
        element.click()
        break

    # 캡차 텍스트 박스에 입력
    chaptchaText = driver.find_element(By.XPATH, "//input[@id='txtCaptcha']")
    chaptchaText.send_keys(captchaValue)

    # 제출 버튼 클릭
    submit_button = driver.find_element(By.CSS_SELECTOR, "div.capchaBtns > a")
    submit_button.click()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.
c:\Users\kty99\AppData\Local\Programs\Python\Python311\Lib\site-packages\easyocr\detection.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an

In [13]:
# 반복 전송을 통해 성공할 때까지 입력하는 작업을 최대 5회 시도
for attempt in range(5):
    process_captcha()
    time.sleep(0.5)

    # "입력한 문자를 다시 확인해주세요" 메시지를 찾기
    error_messages = driver.find_elements(By.XPATH, "//div[@class='alertNotice' and contains(text(), '입력한 문자를 다시 확인해주세요')]")
    if error_messages:
        print("입력한 문자가 일치하지 않습니다. 다시 시도합니다.")
        driver.find_element(By.XPATH, "//a[@class='refreshBtn']").click()
        time.sleep(1)
    else:
        print("에러 메시지가 없습니다.")
        break


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id='imgCaptcha']"}
  (Session info: chrome=129.0.6668.90); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF69207B095+29557]
	(No symbol) [0x00007FF691FEFA50]
	(No symbol) [0x00007FF691EAB56A]
	(No symbol) [0x00007FF691EFF695]
	(No symbol) [0x00007FF691EFF8EC]
	(No symbol) [0x00007FF691F4B777]
	(No symbol) [0x00007FF691F271CF]
	(No symbol) [0x00007FF691F4851C]
	(No symbol) [0x00007FF691F26F33]
	(No symbol) [0x00007FF691EF116F]
	(No symbol) [0x00007FF691EF22D1]
	GetHandleVerifier [0x00007FF6923AC96D+3378253]
	GetHandleVerifier [0x00007FF6923F8497+3688311]
	GetHandleVerifier [0x00007FF6923ED1CB+3642539]
	GetHandleVerifier [0x00007FF69213A6B6+813462]
	(No symbol) [0x00007FF691FFAB5F]
	(No symbol) [0x00007FF691FF6B74]
	(No symbol) [0x00007FF691FF

In [39]:
# 좌석 구역 이름 선택
# driver.find_element(By.XPATH, "InlineTextBox='CU캠핑존(2인)']").click()
# time.sleep(0.5)

# 좌석선택 버튼 클릭
driver.find_element(By.XPATH, '//div[@class="twoBtn"]/a[2]').click()
time.sleep(0.5)

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=129.0.6668.90)
Stacktrace:
	GetHandleVerifier [0x00007FF69207B095+29557]
	(No symbol) [0x00007FF691FEFA50]
	(No symbol) [0x00007FF691EAB399]
	(No symbol) [0x00007FF691F00B0F]
	(No symbol) [0x00007FF691EF354E]
	(No symbol) [0x00007FF691F2718A]
	(No symbol) [0x00007FF691EF2E86]
	(No symbol) [0x00007FF691F273A0]
	(No symbol) [0x00007FF691F4851C]
	(No symbol) [0x00007FF691F26F33]
	(No symbol) [0x00007FF691EF116F]
	(No symbol) [0x00007FF691EF22D1]
	GetHandleVerifier [0x00007FF6923AC96D+3378253]
	GetHandleVerifier [0x00007FF6923F8497+3688311]
	GetHandleVerifier [0x00007FF6923ED1CB+3642539]
	GetHandleVerifier [0x00007FF69213A6B6+813462]
	(No symbol) [0x00007FF691FFAB5F]
	(No symbol) [0x00007FF691FF6B74]
	(No symbol) [0x00007FF691FF6D10]
	(No symbol) [0x00007FF691FE5C1F]
	BaseThreadInitThunk [0x00007FFA331E257D+29]
	RtlUserThreadStart [0x00007FFA33DCAF08+40]


In [41]:
def select_seat():
    # 좌석 요소 가져오기
    seat_elements = driver.find_elements(By.CSS_SELECTOR, '.stySeat')
    seat_info_list = []

    # 좌석 정보 얻기 및 파싱
    for seat in seat_elements:
        seat_title = seat.get_attribute('title')
        print(f"좌석 정보: {seat_title}")

        try:
            seat_row, seat_number_str = seat_title.split('-')
            seat_row = seat_row.strip().replace("열", "")  # "열" 제거
            seat_number = int(seat_number_str.strip())
            seat_info_list.append((seat_row, seat_number, seat))
        except ValueError:
            print(f"좌석 정보를 파싱하는 중 오류 발생: {seat_title}")
            continue

    # 연속 확인 및 선택
    for i, (seat_row, seat_number, seat) in enumerate(seat_info_list):
        if 6 <= seat_number <= 17:  # 6~17 칸 사이의 좌석만 선택
            for j in range(i + 1, len(seat_info_list)):
                next_seat_row, next_seat_number, next_seat = seat_info_list[j]
                if next_seat_row == seat_row and (next_seat_number == seat_number + 1 or next_seat_number == seat_number - 1):
                    # 연속이나 선택된 좌석 목록에 추가
                    try:
                        seat.click()
                        next_seat.click()
                        print(f"연속된 좌석 선택 성공: {seat_row}-{seat_number}, {next_seat_row}-{next_seat_number}")
                        return  # 연속된 좌석을 찾았으므로 함수 종료
                    except Exception as e:
                        print(f"좌석 선택 중 오류 발생: {e}")
                        continue

    # 선택 완료를 위한 대기 시간
    time.sleep(1)


In [42]:
def handle_alert_and_retry():
    try:
        # alert이 뜨면 닫기
        WebDriverWait(driver, 2).until(EC.alert_is_present())
        alert = driver.switch_to.alert
        alert.accept()

        # alert을 처리한 후 다시 시도
        driver.switch_to.default_content()
        driver.switch_to.frame(driver.find_element(By.ID, 'ifrmSeatDetail'))
        select_seat()
    except TimeoutError:
        print("alert이 발생하지 않았습니다")

try:
    select_seat()
    handle_alert_and_retry()
finally:
    print("반복문 끝")

반복문 끝


TimeoutException: Message: 
